# 1. Retriever

In [186]:
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.embeddings import BedrockEmbeddings 
bedrock_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1")

model_kwargs = { 
    "max_tokens_to_sample": 1024, 
    "temperature": 0, 
    "top_p": 0.9, 
    "stop_sequences": ["Human:"]
}

default_model_id = "anthropic.claude-instant-v1"
bedrock_base_kwargs = dict(model_id=default_model_id, model_kwargs= model_kwargs)


In [187]:
persist_directory = 'docs/chroma/'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=bedrock_embeddings)

In [188]:
print(results[0].page_content)

científica y tecnológica, la creación artística y la protección e incremento del 
patrimonio cultural de la Nación.  
 
l) Es deber del Estado y de toda comunidad educativa promover el desarrollo 
profesional y respeto de los docentes y asistentes de la educación.  
 
24. La libertad de enseñanza.  
 
a) La libertad de enseñanza comprende el derecho de las personas de abrir, organizar y 
mantener establecimientos educacionales, así como de crear y desarrol lar proyectos e 
idearios educativos, sin otras limitaciones que las impuestas por la moral, el orden 
público y la seguridad del país.  
 
b) La libertad de enseñanza existe para garantizar a las familias, a través de los padres 
o tutores legales, según sea el caso, el derecho preferente y el deber de educar a sus 
hijos o pupilos; de escoger el tipo de educación; y de enseñarles por sí mismos  o de 
elegir para ellos el establecimiento de enseñanza que estimen de acuerdo con sus


In [189]:
print(vectordb._collection.count())
results = vectordb.max_marginal_relevance_search(query="eixste protección a la libertad de educar?", k=2)

418


# 2. Chatbot

In [190]:
from langchain.chat_models import BedrockChat
llm = BedrockChat(**bedrock_base_kwargs)

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)

In [191]:
result = qa_chain({"query": "eixste protección a la libertad de educar?"})
print(result['result'])

 Sí, en el contexto proporcionado existe protección a la libertad de educar:

- La libertad de enseñanza comprende el derecho de las personas de abrir, organizar y mantener establecimientos educacionales, así como de crear y desarrollar proyectos e idearios educativos (artículo 24, letra a). 

- La libertad de enseñanza existe para garantizar a las familias, a través de los padres o tutores legales, según sea el caso, el derecho preferente y el deber de educar a sus hijos o pupilos; de escoger el tipo de educación; y de enseñarles por sí mismos o de elegir para ellos el establecimiento de enseñanza que estimen de acuerdo con sus convicciones morales o religiosas (artículo 24, letra b).

- Se reconoce la autonomía y la diversidad de proyectos educativos en todos los niveles de enseñanza (artículo 24, letra f).

Por lo tanto, sí existe protección explícita a la libertad de educar y de elegir el tipo de educación según las convicciones de las familias, según lo establecido en este context

# 3. with Memory

In [116]:
from langchain.prompts import PromptTemplate
template = """
You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Assistant:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)


In [97]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)
memory.ai_prefix  = "A:"
memory.human_prefix  = "H:"

In [98]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(search_type = "mmr", similarity_top_k=5 ),
    #return_source_documents=True,
    memory = memory, 
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
)

In [99]:
result = qa_chain({"query": "esta protegido el derecho a la educación?"})
print(result["result"])

 Sí, el derecho a la educación está protegido en la nueva constitución propuesta.

En particular, se encuentra protegido en:

Artículo 24, letra a): Reconoce la libertad de enseñanza, que comprende el derecho a abrir, organizar y mantener establecimientos educacionales, así como crear y desarrollar proyectos e idearios educativos. 

Artículo 24, letra b): Establece que la libertad de enseñanza existe para garantizar a las familias el derecho preferente y el deber de educar a sus hijos o elegir el tipo y establecimiento de educación.

Artículo 24, letra i): Señala que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos los niveles de enseñanza.

Espero haber podido responder de manera clara su consulta sobre la protección del derecho a la educación en la nueva constitución. Validé si mi respuesta fue útil para usted.


In [100]:
result = qa_chain({"query": "y al emprendimiento?"})
print(result["result"])

 Buenas, según los antecedentes provistos:

- El artículo 33 de la propuesta constitucional establece el deber del Estado de no discriminar arbitrariamente en el trato económico. 

- El artículo 21d establece que es deber del Estado promover el emprendimiento y la innovación en las actividades productivas. 

Por lo tanto, de acuerdo a estos artículos, el Estado debe promover el emprendimiento de forma no discriminatoria, es decir, debe fomentar el desarrollo de emprendimientos de forma igualitaria sin distinciones arbitrarias. 

Espero haber podido responder de forma útil su consulta. Por favor, haga llegar cualquier otra duda que surja.


In [171]:
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
    ai_prefix= "A",
    human_prefix="H"
)

retriever=vectordb.as_retriever(search_type = "mmr", similarity_top_k=5, )
qa = ConversationalRetrievalChain.from_llm(
    llm,
    verbose=True,
    retriever=retriever,
    # condense_question_prompt=QA_CHAIN_PROMPT,
    #return_source_documents=True,
    #combine_docs_chain_kwargs={"prompt": QA_CHAIN_PROMPT},
    memory=memory,
)

In [172]:
template = """
You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context", "question"],template=template,)
qa.combine_docs_chain.llm_chain.prompt = QA_CHAIN_PROMPT

In [173]:
print(qa.question_generator.prompt.template)

Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:


In [174]:
print(qa.combine_docs_chain.llm_chain.prompt.template)


You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

{context}
Question: {question}
Helpful Answer:


In [178]:
qa.run({"question": "se protege el derecho a la educación?"})



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?
Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos los niveles.

- Artículo 24, letra l) señala que es deber del Estado y comunidad educativa promover el desarrollo profesional de docentes.

Espero haber podido

/Users/ensamblador/Documents/nueva-constitucion-2023/.venv/lib/python3.11/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?

Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos l


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

16 
  
Solo en virtud de una ley de quorum  calificado, y siempre que no signifique tal 
discriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de 
algún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a 
uno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de 
estos deberá incluirse anualmente en la Ley

' Sí, la nueva constitución protege el derecho al emprendimiento.\n\nEn el artículo 16, letra 34, se establece la "libertad para adquirir el dominio de toda clase de bienes, excepto aquellos que la naturaleza ha hecho comunes a todas las personas o que deban pertenecer a la Nación toda y la ley lo declare así". Esto incluye la libertad para emprender negocios y adquirir bienes necesarios para ello. \n\nAsimismo, en el mismo artículo y letra, se indica que "una ley de quorum calificado, cuando así lo exija el interés nacional, podrá establecer limitaciones o requisitos para la adquisición del dominio de algunos bienes". Esto significa que el emprendimiento está protegido, a menos que exista una ley especial que establezca restricciones por el interés nacional.\n\nEspero haber respondido de manera útil su consulta sobre la protección del derecho al emprendimiento en la nueva constitución. Por favor, indíqueme si requiere alguna aclaración adicional.'

In [175]:
result = qa({"question": "se protege el derecho a la educación?"})
print (result['answer'])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

científica y tecnológica, la creación artística y la protección e incremento del 
patrimonio cultural de la Nación.  
 
l) Es deber del Estado y de toda comunidad educativa promover el desarrollo 
profesional y respeto de los docentes y asistentes de la educación.  
 
24. La libertad de enseñanza.  
 
a) La libertad de enseñanza comprende el derecho de las personas de abrir, organizar y 
mantener establecimientos educacionales, así 

In [176]:
result = qa({"question": "y al emprendimiento?"})
print (result['answer'])



> Entering new LLMChain chain...
Prompt after formatting:
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?
Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos los niveles.

- Artículo 24, letra l) señala que es deber del Estado y comunidad educativa promover el desarrollo profesional de docentes.

Espero haber podido

/Users/ensamblador/Documents/nueva-constitucion-2023/.venv/lib/python3.11/site-packages/langchain/llms/bedrock.py:50: UserWarning: Error: Prompt must alternate between '

Human:' and '

Assistant:'. Received 

Human: Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:

Human: se protege el derecho a la educación?

Assistant:  Sí, la nueva constitución protege el derecho a la educación. En particular:

- Artículo 24, letra a) establece la libertad de enseñanza, que comprende el derecho a abrir establecimientos educacionales y desarrollar proyectos e idearios educativos.

- Artículo 24, letra b) señala que la libertad de enseñanza existe para garantizar a las familias el derecho a educar a sus hijos y elegir el tipo de educación.  

- Artículo 24, letra i) establece que el Estado deberá sostener y coordinar una red pluralista de establecimientos educacionales de calidad en todos l


> Finished chain.


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are a constitutional proffessor who answers questions to users (chilean citizens) about the new constitutional draft to be voted in december.
Use the following pieces of context to answer the question at the end. provide page number, article and letter if it is part of the answer.
If you don't know the answer, please propose a new question rephrasing, don't try to make up an answer.
Always validate at the end if the answer was helpful to the user.

16 
  
Solo en virtud de una ley de quorum  calificado, y siempre que no signifique tal 
discriminación, se podrán autorizar determinados beneficios directos o indirectos en favor de 
algún sector, actividad o zona geográfica, o establecer gravámenes especiales que afecten a 
uno u otras. En el caso de las franquicias o beneficios  indirectos, la estimación del costo de 
estos deberá incluirse anualmente en la Ley

In [179]:
from langchain.chains.question_answering import load_qa_chain

In [ ]:
chain = load_qa_chain(BedrockChat, chain_type="stuff", memory=memory, verbose=True,)